In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [ ]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [ ]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [ ]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

In [ ]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [ ]:
flightsDF = sqlContext.createDataFrame(flights)

In [ ]:
flightsDF.createOrReplaceTempView("flight")

In [ ]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

In [ ]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

In [ ]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

### Count the flights that departed early and arrived late

### Find the flightwith the longest arrival delay

### Find the top 10 destinations ordered by the number of flights

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

### For all flights from JFK during June, show the average departure delay for each destination

### For every origin/dest pair, count the number of flights